In [1]:
# Atividade Prática – Manipulação de Dados com Pandas

## Análise de Filmes - Dataset IMDb

#### Objetivo: Analisar um conjunto de filmes usando DataFrames do Pandas para responder perguntas sobre desempenho, gêneros, diretores e características das produções.

In [2]:
# Importação das bibliotecas necessárias
import pandas as pd
import numpy as np
import re

## 1. Carregamento e Inspeção

### Objetivos desta seção:
- Carregar o arquivo CSV em um DataFrame
- Explorar a estrutura básica dos dados
- Identificar valores ausentes

In [3]:
# 1.1 Carregamento do arquivo CSV
# TODO: Carregue o arquivo 'imdb.csv' em um DataFrame chamado 'df'
df = pd.read_csv("./datasets/imdb.csv")
# TODO: Exiba informações básicas sobre o dataset (número de linhas, colunas)
print(df.info())       # Tipos das colunas e valores nulos
print(df.head())       # Primeiras linhas
print(df.describe())   # Estatísticas básicas (para colunas numéricas)
# TODO: Liste os nomes das colunas disponíveis
df.columns

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33600 entries, 0 to 33599
Data columns (total 23 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     33600 non-null  object 
 1   title                  33600 non-null  object 
 2   link                   33600 non-null  object 
 3   year                   33600 non-null  int64  
 4   duration               33379 non-null  object 
 5   rating_mpa             25624 non-null  object 
 6   rating_imdb            33462 non-null  float64
 7   vote                   33462 non-null  float64
 8   budget                 11815 non-null  float64
 9   gross_world_wide       18222 non-null  float64
 10  gross_us_canada        17571 non-null  float64
 11  gross_opening_weekend  15523 non-null  float64
 12  director               33241 non-null  object 
 13  writer                 32024 non-null  object 
 14  star                   33127 non-null  object 
 15  ge

Index(['id', 'title', 'link', 'year', 'duration', 'rating_mpa', 'rating_imdb',
       'vote', 'budget', 'gross_world_wide', 'gross_us_canada',
       'gross_opening_weekend', 'director', 'writer', 'star', 'genre',
       'country_origin', 'filming_location', 'production_company', 'language',
       'win', 'nomination', 'oscar'],
      dtype='object')

In [4]:
# 1.2 Exibição das primeiras linhas e informações básicas
# TODO: Use df.head() para exibir as primeiras 5 linhas
print(df.head())
# TODO: Use df.info() para exibir informações gerais sobre o DataFrame
print(df.info())
# TODO: Use df.describe() para exibir estatísticas descritivas
print(df.describe())

          id                               title  \
0  tt0073470          Strip Nude for Your Killer   
1  tt0072764                     Carry on Behind   
2  tt0131526  So Young, So Lovely, So Vicious...   
3  tt0073335                            Mahogany   
4  tt0073006                            Foreplay   

                                   link  year duration rating_mpa  \
0  https://www.imdb.com/title/tt0073470  1975   1h 38m  Not Rated   
1  https://www.imdb.com/title/tt0072764  1975   1h 30m  Not Rated   
2  https://www.imdb.com/title/tt0131526  1975   1h 30m        NaN   
3  https://www.imdb.com/title/tt0073335  1975   1h 49m         PG   
4  https://www.imdb.com/title/tt0073006  1975   1h 15m          R   

   rating_imdb     vote  budget  gross_world_wide  ...  \
0          5.6  33000.0     NaN               NaN  ...   
1          5.6  31000.0     NaN               NaN  ...   
2          4.9    487.0     NaN               NaN  ...   
3          6.1  23000.0     NaN         

In [5]:
# 1.3 Identificação de valores ausentes
# TODO: Use df.isnull().sum() para contar valores ausentes por coluna
print("📌 Valores ausentes por coluna:")
print(df.isnull().sum())
# TODO: Calcule o percentual de valores ausentes
percentual_nulos = (df.isnull().sum() / len(df)) * 100

print("📌 Percentual de valores ausentes por coluna:")
print(percentual_nulos)
# TODO: Crie um DataFrame organizado mostrando colunas com valores ausentes
#Contagem de valores ausentes
nulos = df.isnull().sum()

# Percentual de valores ausentes
percentual = (nulos / len(df)) * 100

# Criar DataFrame apenas com colunas que têm nulos
missing_report = pd.DataFrame({
    "Valores Nulos": nulos,
    "Percentual (%)": percentual
})

# Filtrar colunas com pelo menos 1 valor ausente
missing_report = missing_report[missing_report["Valores Nulos"] > 0]

# Ordenar do maior para o menor percentual
missing_report = missing_report.sort_values(by="Percentual (%)", ascending=False)

print("📌 Relatório de colunas com valores ausentes:")
print(missing_report)

📌 Valores ausentes por coluna:
id                           0
title                        0
link                         0
year                         0
duration                   221
rating_mpa                7976
rating_imdb                138
vote                       138
budget                   21785
gross_world_wide         15378
gross_us_canada          16029
gross_opening_weekend    18077
director                   359
writer                    1576
star                       473
genre                      382
country_origin             366
filming_location          6729
production_company        1378
language                   491
win                          0
nomination                   0
oscar                        0
dtype: int64
📌 Percentual de valores ausentes por coluna:
id                        0.000000
title                     0.000000
link                      0.000000
year                      0.000000
duration                  0.657738
rating_mpa             

## 2. Limpeza e Transformação

### Objetivos desta seção:
- Converter colunas para tipos numéricos apropriados
- Criar nova coluna de lucro
- Padronizar nomes das colunas

In [6]:
# 2.1 Conversão de tipos de dados

# TODO: Crie uma função para converter duration (ex: "1h 38m" -> 98 minutos)
def convert_duration(duration_str):
    """
    Converte string de duração no formato 'Xh Ym' para minutos (int).
    Exemplo: '1h 38m' -> 98
    """
    if pd.isnull(duration_str):  # caso seja NaN
        return None
    
    # Regex para capturar horas e minutos
    match = re.match(r'(?:(\d+)h)?\s*(?:(\d+)m)?', duration_str.strip())
    if not match:
        return None
    
    hours = int(match.group(1)) if match.group(1) else 0
    minutes = int(match.group(2)) if match.group(2) else 0
    
    return hours * 60 + minutes



# TODO: Aplique as conversões usando pd.to_numeric() com errors='coerce':
    # Converter a coluna duration para minutos usando a função criada
df["duration_minutes"] = df["duration"].apply(convert_duration)

# Aplicar conversões numéricas com pd.to_numeric (valores inválidos viram NaN)
df["year"] = pd.to_numeric(df["year"], errors="coerce")
df["rating_imdb"] = pd.to_numeric(df["rating_imdb"], errors="coerce")
df["vote"] = pd.to_numeric(df["vote"], errors="coerce")
df["budget"] = pd.to_numeric(df["budget"], errors="coerce")
df["gross_world_wide"] = pd.to_numeric(df["gross_world_wide"], errors="coerce")
df["gross_us_canada"] = pd.to_numeric(df["gross_us_canada"], errors="coerce")
df["oscar"] = pd.to_numeric(df["oscar"], errors="coerce")

# Exibir amostra para verificar conversões
print("📌 Amostra das conversões realizadas:")
print(df[["year", "duration", "duration_minutes", 
          "rating_imdb", "vote", "budget", 
          "gross_world_wide", "gross_us_canada", "oscar"]].head())

# TODO: Exiba uma amostra das conversões realizadas

📌 Amostra das conversões realizadas:
   year duration  duration_minutes  rating_imdb     vote  budget  \
0  1975   1h 38m              98.0          5.6  33000.0     NaN   
1  1975   1h 30m              90.0          5.6  31000.0     NaN   
2  1975   1h 30m              90.0          4.9    487.0     NaN   
3  1975   1h 49m             109.0          6.1  23000.0     NaN   
4  1975   1h 15m              75.0          4.0    185.0     NaN   

   gross_world_wide  gross_us_canada  oscar  
0               NaN              NaN      0  
1               NaN              NaN      0  
2               NaN              NaN      0  
3               NaN        5000000.0      1  
4               NaN              NaN      0  


In [7]:
# 2.2 Criação da coluna profit
# TODO: Crie a coluna 'profit' = gross_world_wide - budget
# Criar a coluna 'profit'
df["profit"] = df["gross_world_wide"] - df["budget"]



# TODO: Exiba estatísticas da nova coluna (quantos filmes têm dados de lucro, etc.)
# Exibir estatísticas descritivas da nova coluna

print("📌 Estatísticas da coluna 'profit':")
print(df["profit"].describe())

# Quantos filmes têm valores válidos de lucro
print("\n📌 Quantidade de filmes com dado de lucro:", df["profit"].notnull().sum())

# Quantos têm lucro positivo, negativo e zero
print("\n📌 Distribuição de lucro:")
print("Lucro positivo:", (df["profit"] > 0).sum())
print("Lucro negativo:", (df["profit"] < 0).sum())
print("Lucro zero:", (df["profit"] == 0).sum())



📌 Estatísticas da coluna 'profit':
count    9.406000e+03
mean    -3.386014e+07
std      3.214523e+09
min     -2.999844e+11
25%     -4.735026e+06
50%      3.834980e+05
75%      3.102142e+07
max      2.686706e+09
Name: profit, dtype: float64

📌 Quantidade de filmes com dado de lucro: 9406

📌 Distribuição de lucro:
Lucro positivo: 4930
Lucro negativo: 4476
Lucro zero: 0


In [8]:
# 2.3 Padronização dos nomes das colunas
# TODO: Padronize os nomes das colunas (minúsculas, sem espaços, substituir por _)
df.columns = (
    df.columns
    .str.strip()            # remove espaços extras no início/fim
    .str.lower()            # tudo minúsculo
    .str.replace(" ", "_")  # troca espaços por underscore
    .str.replace("-", "_")  # troca hífen por underscore (se existir)
)


# TODO: Exiba os nomes das colunas após padronização
print("📌 Novos nomes das colunas:")
print(df.columns.tolist())

📌 Novos nomes das colunas:
['id', 'title', 'link', 'year', 'duration', 'rating_mpa', 'rating_imdb', 'vote', 'budget', 'gross_world_wide', 'gross_us_canada', 'gross_opening_weekend', 'director', 'writer', 'star', 'genre', 'country_origin', 'filming_location', 'production_company', 'language', 'win', 'nomination', 'oscar', 'duration_minutes', 'profit']


## 3. Exploração e Agregação

### Objetivos desta seção:
- Identificar filmes com maiores notas IMDb
- Calcular estatísticas por década
- Encontrar o melhor diretor
- Identificar gênero mais comum

In [9]:
# 3.1 Top 5 filmes com maior nota IMDb
# TODO: Use sort_values() para ordenar por 'rating_imdb' (decrescente)
top5_rating = df.sort_values(by="rating_imdb", ascending=False).head(5)

print("📌 Top 5 filmes com maior nota IMDb:")
print(top5_rating[["title", "rating_imdb", "year"]])


#Alterado modelo GPT-5 para Gemini 2.5 Pro


# TODO: Selecione os primeiros 5 filmes
primeiros_5_filmes = df.head(5)

# TODO: Exiba colunas relevantes: title, year, rating_imdb, vote, director, genre
colunas_relevantes = ['title', 'year', 'rating_imdb', 'vote', 'director', 'genre']
resultado = primeiros_5_filmes[colunas_relevantes]
print(resultado)

📌 Top 5 filmes com maior nota IMDb:
                                                title  rating_imdb  year
11166                         Day by Day: The Dynasty          9.6  2023
20986                                Love Goes Public          9.6  2019
12225                                   Water Brother          9.5  2024
14532  SEVENTEEN TOUR 'FOLLOW' to JAPAN: LIVE VIEWING          9.5  2023
27302                        The Cowboy and the Queen          9.4  2023
                                title  year  rating_imdb     vote  \
0          Strip Nude for Your Killer  1975          5.6  33000.0   
1                     Carry on Behind  1975          5.6  31000.0   
2  So Young, So Lovely, So Vicious...  1975          4.9    487.0   
3                            Mahogany  1975          6.1  23000.0   
4                            Foreplay  1975          4.0    185.0   

                                         director  \
0                                  Andrea Bianchi   
1    

In [10]:
# 3.2 Média de duração por década
# TODO: Crie uma coluna 'decade' usando (year // 10) * 10
df['decade'] = (df['year'] // 10) * 10

# TODO: Use groupby('decade') para calcular a média de 'duration_minutes'
media_por_decada = df.groupby('decade')['duration_minutes'].mean()

# TODO: Exiba os resultados
print("Média de duração por década:")
print(media_por_decada)

# TODO EXTRA: Calcule outras estatísticas por década
estatisticas_por_decada = df.groupby('decade').agg(
    media_rating_imdb=('rating_imdb', 'mean'),
    media_votos=('vote', 'mean'),
    quantidade_filmes=('title', 'count')
)

# Exibindo os resultados
print("Estatísticas por década:")
print(estatisticas_por_decada)


Média de duração por década:
decade
1960     99.709957
1970    100.384062
1980    101.817836
1990    103.887324
2000    103.673278
2010    106.652791
2020    107.420254
Name: duration_minutes, dtype: float64
Estatísticas por década:
        media_rating_imdb    media_votos  quantidade_filmes
decade                                                     
1960             6.179788   12085.734954               5149
1970             5.935427   27782.298211               5050
1980             5.913377   36688.839197               5050
1990             6.047076  107496.141570               5001
2000             6.335423  110709.429266               5349
2010             6.457151  114801.467707               5151
2020             6.234367   41643.711315               2850


In [11]:
# 3.3 Diretor com maior média de nota IMDb (mínimo 3 filmes)
# TODO: Use groupby('director') e calcule as estatísticas
diretores_stats = df.groupby('director').agg(
    rating_imdb_mean=('rating_imdb', 'mean'),
    movie_count=('title', 'count'),
    movie_titles=('title', list)
)

# TODO: Filtre apenas diretores com 3 ou mais filmes
diretores_filtrados = diretores_stats[diretores_stats['movie_count'] >= 3]

# TODO: Ordene por nota média (decrescente)
diretores_top = diretores_filtrados.sort_values(by='rating_imdb_mean', ascending=False)

# TODO: Exiba o top 10
print("Top 10 Diretores por Média de Nota IMDb (mínimo de 3 filmes):")
print(diretores_top.head(10))

# TODO: Identifique o melhor diretor
if not diretores_top.empty:
    melhor_diretor = diretores_top.index[0]
    nota_media = diretores_top['rating_imdb_mean'].iloc[0]
    print(f"\nO melhor diretor, com base neste dataset, é {melhor_diretor} com uma nota média de {nota_media:.2f}.")
else:
    print("\nNenhum diretor com 3 ou mais filmes encontrado no dataset.")



Top 10 Diretores por Média de Nota IMDb (mínimo de 3 filmes):
                           rating_imdb_mean  movie_count  \
director                                                   
Jim Brown                          8.633333            3   
Sachin Pilgaonkar                  8.533333            3   
Upendra                            8.514286            7   
Slobodan Sijan                     8.500000            3   
Kamalakara Kameshwara Rao          8.433333            3   
Mircea Dragan                      8.400000            5   
Moustapha Akkad                    8.400000            3   
Sam Wrench                         8.400000            3   
Bharathan                          8.300000            4   
Srdjan Dragojevic                  8.300000            3   

                                                                movie_titles  
director                                                                      
Jim Brown                  [The Weavers: Wasn't That a Time

In [12]:
# 3.4 Gênero mais recorrente no dataset
# TODO: Separar gêneros combinados e criar uma lista com todos os gêneros individuais
# O método .dropna() remove filmes sem gênero e .str.split(', ') separa a string em uma lista de gêneros
# O método .explode() transforma cada gênero da lista em uma linha separada
todos_os_generos = df['genre'].dropna().str.split(', ').explode()

# TODO: Use pd.Series().value_counts() para contar frequências
contagem_generos = todos_os_generos.value_counts()

# TODO: Exiba o top 10 gêneros com percentuais
total_generos = len(todos_os_generos)
top_10_generos = contagem_generos.head(10)

print("Top 10 Gêneros Mais Recorrentes:")
for genero, contagem in top_10_generos.items():
    percentual = (contagem / total_generos) * 100
    print(f"- {genero}: {contagem} filmes ({percentual:.2f}%)")

# TODO: Identifique o gênero mais recorrente
genero_mais_recorrente = contagem_generos.index[0]
print(f"\nO gênero mais recorrente no dataset é '{genero_mais_recorrente}'.")

Top 10 Gêneros Mais Recorrentes:
- Drama: 18343 filmes (17.64%)
- Comedy: 11021 filmes (10.60%)
- Thriller: 6873 filmes (6.61%)
- Romance: 6451 filmes (6.20%)
- Action: 5472 filmes (5.26%)
- Crime: 5180 filmes (4.98%)
- Horror: 4024 filmes (3.87%)
- Adventure: 3870 filmes (3.72%)
- Mystery: 2835 filmes (2.73%)
- Fantasy: 2591 filmes (2.49%)

O gênero mais recorrente no dataset é 'Drama'.


## 4. Joins e Relações

### Objetivos desta seção:
- Criar DataFrame separado com informações de Oscar
- Realizar merge entre DataFrames
- Filtrar filmes ganhadores de Oscar

In [13]:
# 4.1 Carregue o arquivo 'oscars.csv' em um DataFrame chamado 'oscars'.
# Certifique-se de utilizar a codificação correta ao ler o arquivo.
# Define o nome do arquivo para facilitar a manutenção
file_path = './datasets/oscars.csv'
oscars = pd.read_csv(file_path, delimiter="\t")
print("Arquivo 'oscars.csv' carregado com sucesso usando a codificação UTF-8.")

print(oscars.head())




Arquivo 'oscars.csv' carregado com sucesso usando a codificação UTF-8.
   Ceremony     Year   Class          CanonicalCategory Category  \
0         1  1927/28  Acting    ACTOR IN A LEADING ROLE    ACTOR   
1         1  1927/28  Acting    ACTOR IN A LEADING ROLE    ACTOR   
2         1  1927/28  Acting  ACTRESS IN A LEADING ROLE  ACTRESS   
3         1  1927/28  Acting  ACTRESS IN A LEADING ROLE  ACTRESS   
4         1  1927/28  Acting  ACTRESS IN A LEADING ROLE  ACTRESS   

                                    Film                         FilmId  \
0       The Noose|The Patent Leather Kid            tt0019217|tt0018253   
1  The Last Command|The Way of All Flesh            tt0019071|tt0019553   
2                        A Ship Comes In                      tt0018389   
3        7th Heaven|Street Angel|Sunrise  tt0018379|tt0019429|tt0018455   
4                         Sadie Thompson                      tt0019344   

                  Name             Nominees NomineeIds Winner  \
0  R

In [14]:
# 4.2 Realize o merge entre o DataFrame principal (df) e o DataFrame 'oscars',
# --- 1. Limpeza do DataFrame 'oscars' ---

# Converte as strings em 'FilmId' e 'Film' em listas, dividindo pelo caractere '|'
oscars['FilmId'] = oscars['FilmId'].str.split('|')
oscars['Film'] = oscars['Film'].str.split('|')

# "Explode" o DataFrame para que cada filme em uma lista tenha sua própria linha.
# Passar uma lista de colunas para .explode() garante que os IDs e Títulos correspondentes permaneçam alinhados.
oscars_cleaned = oscars.explode(['FilmId', 'Film'])

print("\n--- DataFrame 'oscars' Limpo (após explode) ---")
print(oscars_cleaned)


# --- 2. Merge com o DataFrame Limpo ---

# Agora, realizamos o merge usando o oscars_cleaned
df_merged = pd.merge(
    df,
    oscars_cleaned,
    how='left',
    left_on=['id', 'title'],
    right_on=['FilmId', 'Film']
)

print("\n--- Resultado Final do Merge ---")
df_merged


--- DataFrame 'oscars' Limpo (após explode) ---
       Ceremony     Year    Class  \
0             1  1927/28   Acting   
0             1  1927/28   Acting   
1             1  1927/28   Acting   
1             1  1927/28   Acting   
2             1  1927/28   Acting   
...         ...      ...      ...   
11988        97     2024  SciTech   
11989        97     2024  SciTech   
11990        97     2024  SciTech   
11991        97     2024  SciTech   
11992        97     2024  SciTech   

                                       CanonicalCategory  \
0                                ACTOR IN A LEADING ROLE   
0                                ACTOR IN A LEADING ROLE   
1                                ACTOR IN A LEADING ROLE   
1                                ACTOR IN A LEADING ROLE   
2                              ACTRESS IN A LEADING ROLE   
...                                                  ...   
11988  SCIENTIFIC AND TECHNICAL AWARD (Technical Achi...   
11989  SCIENTIFIC AND TECH

,id,title,link,year,duration,rating_mpa,rating_imdb,vote,budget,gross_world_wide,...,Category,Film,FilmId,Name,Nominees,NomineeIds,Winner,Detail,Note,Citation
0,tt0073470,Strip Nude for Your Killer,https://www.imdb.com/title/tt0073470,1975,1h 38m,Not Rated,5.6,33000.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,tt0072764,Carry on Behind,https://www.imdb.com/title/tt0072764,1975,1h 30m,Not Rated,5.6,31000.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,tt0131526,"So Young, So Lovely, So Vicious...",https://www.imdb.com/title/tt0131526,1975,1h 30m,NaN,4.9,487.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,tt0073335,Mahogany,https://www.imdb.com/title/tt0073335,1975,1h 49m,PG,6.1,23000.0,NaN,NaN,...,MUSIC (Original Song),Mahogany,tt0073335,Music by Michael Masser; Lyrics by Gerry Goffin,Michael Masser|Gerry Goffin,nm0557247|nm1132342,NaN,Theme From Mahogany (Do You Know Where You're ...,NaN,NaN
4,tt0073006,Foreplay,https://www.imdb.com/title/tt0073006,1975,1h 15m,R,4.0,185.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37172,tt0095428,Kansas,https://www.imdb.com/title/tt0095428,1988,1h 50m,R,5.5,14000.0,NaN,2432536.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37173,tt0094980,The Decline of Western Civilization Part II: T...,https://www.imdb.com/title/tt0094980,1988,1h 33m,R,7.2,45000.0,500000.0,373743.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37174,tt0093170,Man Behind the Sun,https://www.imdb.com/title/tt0093170,1988,1h 45m,Not Rated,6.1,76000.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37175,tt0170783,Yesterday,https://www.imdb.com/title/tt0170783,1988,1h 24m,NaN,8.9,36000.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
df_merged.head().columns

Index(['id', 'title', 'link', 'year', 'duration', 'rating_mpa', 'rating_imdb',
       'vote', 'budget', 'gross_world_wide', 'gross_us_canada',
       'gross_opening_weekend', 'director', 'writer', 'star', 'genre',
       'country_origin', 'filming_location', 'production_company', 'language',
       'win', 'nomination', 'oscar', 'duration_minutes', 'profit', 'decade',
       'Ceremony', 'Year', 'Class', 'CanonicalCategory', 'Category', 'Film',
       'FilmId', 'Name', 'Nominees', 'NomineeIds', 'Winner', 'Detail', 'Note',
       'Citation'],
      dtype='object')

In [24]:
# 4.3 Identifique o diretor com o maior número de prêmios de BEST PICTURE,
# TODO: Filtrar apenas os filmes que ganharam a categoria 'BEST PICTURE'
# Condição 1: A categoria precisa ser 'BEST PICTURE'
condicao_categoria = df_merged['Category'] == 'BEST PICTURE'
# Condição 2: O valor na coluna 'winner' precisa ser True
condicao_vencedor = df_merged['Winner'] == True

# Aplicando ambas as condições para obter o DataFrame final
best_picture_winners = df_merged[condicao_categoria & condicao_vencedor]

# TODO: Contar o número de prêmios por diretor
contagem_diretores = best_picture_winners['director'].value_counts()

# TODO: Exibir o resultado e identificar o diretor com mais prêmios
print("Diretores com mais prêmios de 'BEST PICTURE':")
print(contagem_diretores)

if not contagem_diretores.empty:
    melhor_diretor = contagem_diretores.index[0]
    num_premios = contagem_diretores.iloc[0]
    print(f"\nO diretor com o maior número de prêmios de 'BEST PICTURE' é {melhor_diretor}, com {num_premios} vitórias.")
else:
    print("\nNão foram encontrados diretores vencedores de 'BEST PICTURE' no dataset.")

Diretores com mais prêmios de 'BEST PICTURE':
director
Clint Eastwood                   2
Francis Ford Coppola             2
Sydney Pollack                   1
Robert Zemeckis                  1
John Schlesinger                 1
Robert Wise                      1
Chlo Zhao                        1
Tony Richardson                  1
Peter Jackson                    1
Jonathan Demme                   1
Barry Levinson                   1
James Cameron                    1
George Cukor                     1
James L Brooks                   1
Peter Farrelly                   1
Robert Benton                    1
Woody Allen                      1
Guillermo del Toro               1
Robert Redford                   1
Sean Baker                       1
Carol Reed                       1
Bernardo Bertolucci              1
Daniel Kwan, Daniel Scheinert    1
Milos Forman                     1
David Lean                       1
Christopher Nolan                1
Kathryn Bigelow                  1


## 5. Pivot e Reshaping

### Objetivos desta seção:
- Criar pivot table com dados de rating por país e década
- Usar melt para reorganizar dados de bilheteria

In [25]:
# 5.1 Pivot table: média de rating_imdb por país e década
# TODO: Use pivot_table() para criar a tabela
pivot_ratings = df.pivot_table(
    values='rating_imdb',
    index='country_origin',
    columns='decade',
    aggfunc='mean',
    fill_value=0
)

# TODO: Exiba as dimensões e primeiras linhas da pivot table
print("Dimensões da Tabela Dinâmica (Países, Décadas):", pivot_ratings.shape)
print("\nTabela Dinâmica: Média de Rating IMDb por País e Década")
print(pivot_ratings.head())


# TODO: EXTRA: Identifique os países com melhores médias por década
# O método .idxmax() retorna o índice (neste caso, o país) do valor máximo em cada coluna.
melhores_paises_por_decada = pivot_ratings.idxmax()

print("\nPaís com a melhor média de rating por década:")
print(melhores_paises_por_decada)

Dimensões da Tabela Dinâmica (Países, Décadas): (2937, 7)

Tabela Dinâmica: Média de Rating IMDb por País e Década
decade                                          1960  1970  1980  1990  2000  \
country_origin                                                                 
Afghanistan, Ireland, Japan, Netherlands, Iran   0.0   0.0   0.0   0.0   7.3   
Algeria                                          0.0   7.3   0.0   0.0   0.0   
Algeria, France                                  0.0   0.0   0.0   0.0   5.8   
Algeria, France, Morocco, Belgium                0.0   0.0   0.0   0.0   7.0   
Angola, France                                   0.0   7.0   0.0   0.0   0.0   

decade                                          2010  2020  
country_origin                                              
Afghanistan, Ireland, Japan, Netherlands, Iran   0.0   0.0  
Algeria                                          0.0   0.0  
Algeria, France                                  0.0   0.0  
Algeria, France, Mo

In [27]:
# 5.2 Melt: reorganizar colunas de bilheteria em formato longo
# TODO: Compare shapes antes do melt
print(f"Shape do seu DataFrame 'df' ANTES do melt: {df.shape}")

# TODO: Use melt() para reorganizar as colunas do seu DataFrame 'df'
df_long = pd.melt(
    df,  # <--- Usando o seu DataFrame principal aqui
    id_vars=['id', 'title', 'year', 'director'], # Adicionei 'director' como exemplo de outra coluna identificadora
    value_vars=['gross_us_canada', 'gross_world_wide'],
    var_name='gross_type',
    value_name='gross_value'
)

# TODO: Remova valores nulos da coluna gross_value
df_long_cleaned = df_long.dropna(subset=['gross_value'])

# TODO: Compare shapes depois do melt
print(f"Shape DEPOIS do melt e limpeza: {df_long_cleaned.shape}")
print("\nPrévia do DataFrame Transformado (Formato Longo):")
print(df_long_cleaned.head())

# TODO: Calcule estatísticas por tipo de bilheteria
print("\n--- Análise por Tipo de Bilheteria ---")
estatisticas_bilheteria = df_long_cleaned.groupby('gross_type')['gross_value'].agg(['count', 'mean', 'std'])
print(estatisticas_bilheteria)

Shape do seu DataFrame 'df' ANTES do melt: (33600, 26)
Shape DEPOIS do melt e limpeza: (35793, 6)

Prévia do DataFrame Transformado (Formato Longo):
           id                              title  year  \
3   tt0073335                           Mahogany  1975   
11  tt0076489                           Oh, God!  1977   
12  tt0076138                             Heroes  1977   
13  tt0076591  Race for Your Life, Charlie Brown  1977   
14  tt0076706                         Short Eyes  1977   

                                      director       gross_type  gross_value  
3   Berry Gordy, Tony Richardson, Jack Wormser  gross_us_canada    5000000.0  
11                                 Carl Reiner  gross_us_canada   41687243.0  
12                                Jeremy Kagan  gross_us_canada   33500000.0  
13                   Bill Melendez, Phil Roman  gross_us_canada    3223888.0  
14                              Robert M Young  gross_us_canada       3456.0  

--- Análise por Tipo de Bil

## 6. Exportação

### Objetivo desta seção:
- Salvar tabela final processada em formato CSV

In [28]:
# 6.1 Exportação da tabela pivot
# TODO: Gere a tabela pivot
pivot_ratings = pd.pivot_table(
    df,
    values='rating_imdb',
    index='country_origin',
    columns='decade',
    aggfunc='mean',
    fill_value=0
)

# TODO: Verifique se o DataFrame resultante não está vazio
if not pivot_ratings.empty:
    print("--- Tabela Pivot Gerada ---")
    # TODO: Exiba algumas linhas da tabela pivot para conferência
    print(pivot_ratings.head())

    # Define o nome do arquivo de saída
    filename = './rating_pivot_pais_decada.csv'

    # TODO: Exporte a tabela pivot para CSV
    # Nota: A dica pedia index=False. Isso remove a coluna 'country_origin' do arquivo.
    # Se quisesse manter 'country_origin' como a primeira coluna, usaria index=True.
    pivot_ratings.to_csv(filename, index=True, encoding='utf-8')

    # TODO: Exiba mensagem de confirmação e amostra dos dados exportados
    print(f"\n[SUCESSO] Tabela pivot exportada para o arquivo '{filename}'.")
    
    # Para confirmar, vamos ler o arquivo recém-criado e mostrar uma amostra
    print("\n--- Amostra do Arquivo CSV Exportado ---")
    with open(filename, 'r', encoding='utf-8') as f:
        for _ in range(5): # Lê e exibe as 5 primeiras linhas
            line = f.readline().strip()
            if line:
                print(line)

else:
    print("[AVISO] A tabela pivot resultante está vazia. Nenhum arquivo foi exportado.")


--- Tabela Pivot Gerada ---
decade                                          1960  1970  1980  1990  2000  \
country_origin                                                                 
Afghanistan, Ireland, Japan, Netherlands, Iran   0.0   0.0   0.0   0.0   7.3   
Algeria                                          0.0   7.3   0.0   0.0   0.0   
Algeria, France                                  0.0   0.0   0.0   0.0   5.8   
Algeria, France, Morocco, Belgium                0.0   0.0   0.0   0.0   7.0   
Angola, France                                   0.0   7.0   0.0   0.0   0.0   

decade                                          2010  2020  
country_origin                                              
Afghanistan, Ireland, Japan, Netherlands, Iran   0.0   0.0  
Algeria                                          0.0   0.0  
Algeria, France                                  0.0   0.0  
Algeria, France, Morocco, Belgium                0.0   0.0  
Angola, France                               